In [11]:
import sys, time, re
from random import randint
import cyrtranslit
import os
import pandas as pd
import pylab as plt
from tqdm import tqdm
%matplotlib inline 

In [17]:
import cyrtranslit

def clean_lineinst(line):
    inst_dict = dict()
    pat = re.compile("(\d+),(.*),(.*)")
    pat_word = re.compile('[^a-zA-Zа-яА-Я\d\s]+')

    uid, uname, fname = pat.match(line).groups()
    fname = re.sub(pat_word, '', fname).strip().lower()
    
    fname = cyrtranslit.to_latin(fname, 'ru').replace("'", '')
    return (uid, uname, fname)

def clean_linevk(line):
    inst_dict = dict()
    pat = re.compile("(\d+),(.*),(.*),(.*)")
    pat_word = re.compile('[^a-zA-Zа-яА-Я\d\s]+')

    try:
        uid, uname, name1, name2 = pat.match(line).groups()
        name1 = re.sub(pat_word, '', name1).strip().lower()
        name2 = re.sub(pat_word, '', name2).strip().lower()
        fname = name1 + ' ' + name2
    except AttributeError:
        print(line)
    fname = cyrtranslit.to_latin(fname, 'ru').replace("'", '')
    return (uid, uname, fname)

folder = 'data'

def read_clean_csv(fname, num_col):
    df = pd.DataFrame()

    clean_line = clean_lineinst if num_col == 3 else clean_linevk
    with open(os.path.join(folder, fname), 'r') as f:
        for line in tqdm(f.readlines()):
            df = pd.concat( [df, pd.DataFrame([clean_line(line)])], ignore_index=True )
    return df

In [18]:
inst = read_clean_csv(fname = 'inst_personal.csv', num_col = 3)
inst.columns = ['uid', 'uname', 'inst_name']
inst.head()

100%|██████████| 24859/24859 [00:46<00:00, 530.15it/s]


,uid,uname,inst_name
0,751026638,mrzelkin1,aleksej zelkin
1,2744159,undruha,andrey gnelitskiy
2,4157858483,vadimbhai,abhairov vadim
3,389335890,superov_sergey,sergey superov
4,199238507,smilychka,nastja gogol


In [21]:
vk = read_clean_csv(fname = 'vk_personal.csv', num_col = 4)
vk.columns = ['uid', 'uname', 'vk_name']
vk.head()

100%|██████████| 26224/26224 [00:54<00:00, 478.66it/s]


,uid,uname,vk_name
0,5394,mrzelkin1,alexey zelkin
1,22884,undruha,andre undrukhov
2,23754,vadimbhai,vadim reutsky
3,89831,superov_sergey,sergey superov
4,103177,smilychka,nastya gogol


In [37]:
inst_uname_id = dict(((d[1], d[0]) for d in inst[['uid', 'uname']].values))
vk_id_name = dict(((d[0], d[1]) for d in vk[['uid', 'uname']].values))

In [72]:
def get_sim(uid):
    return inst_uname_id[vk_id_name[uid]] if uid in vk_id_name and vk_id_name[uid] in inst_uname_id else None

def generate_overlap(df):
    ls = []
    with open(os.path.join(folder, 'vk_lid_rid.csv'), 'r') as f:
        for line in f.readlines():
            l, r = line.split()
            l = get_sim(l)
            r = get_sim(r)
            if l and r:
                ls.append((l, r))
    return ls

In [73]:
with open(os.path.join(folder, 'same_to_inst_vk.csv'), 'w') as rw:
    ls = generate_overlap(vk)
    for tup in ls:
        rw.write('%s %s\n' % tup)